In [1]:
#import dependencies

import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
#from config import username, password
import psycopg2

from config import username
from config import password

In [2]:
#read economic data data file

wv1 = pd.read_csv("Resources\WV1.csv",header=[0,3])
wv1.head()

,Unnamed: 0_level_0,Population,Surface area,Population density,"Gross national income, Atlas method","Gross national income per capita, Atlas method",Purchasing power parity gross national income,Unnamed: 7_level_0,Gross domestic product,Unnamed: 9_level_0
,Unnamed: 0_level_1,2019,2019,2019,2019,2019,2019,2019,2019,2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [3]:
wv1.columns=wv1.columns.map("_".join)
wv1.head()

,Unnamed: 0_level_0_Unnamed: 0_level_1,Population_2019,Surface area_2019,Population density_2019,"Gross national income, Atlas method_2019","Gross national income per capita, Atlas method_2019",Purchasing power parity gross national income_2019,Unnamed: 7_level_0_2019,Gross domestic product_2019,Unnamed: 9_level_0_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [4]:
#clean header names

wv1.columns = [x.lower().replace(" ","_").replace(",","") for x in wv1.columns]
                    
wv1.head()

,unnamed:_0_level_0_unnamed:_0_level_1,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,unnamed:_7_level_0_2019,gross_domestic_product_2019,unnamed:_9_level_0_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [5]:
wv1_df=wv1.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_7_level_0_2019":"ppp_gni_per_capita_2019",\
                          "gross_domestic_product_2019":"gross_domestic_product_growth_2019", "unnamed:_9_level_0_2019":"per_capita_gdp_growth_2019"})
wv1_df.head()

,country,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,ppp_gni_per_capita_2019,gross_domestic_product_growth_2019,per_capita_gdp_growth_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7


In [6]:
#read poverty data file

wv2 = pd.read_csv("Resources\WV2.csv",header=[0,3])
wv2.head()

,Unnamed: 0_level_0,Percentage share of income or consumption,Prevalence of child malnutrition,Maternal mortality ratio,Under-five mortality rate,"Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)",Incidence of tuberculosis,Mortality caused by road traffic injury,Primary completion rate,Vulnerable employment,Unnamed: 10_level_0,Labor productivity
,Unnamed: 0_level_1,2007-18,2020,2017,2019,2020,2019,2016,2018,2019,2019,2015-18
0,Afghanistan,..,35.1,638,60,0.08,189,15,86,76.7,89.5,-2
1,Albania,7.5,9.6,15,10,0.05,16,14,102,51.3,51.1,-0.6
2,Algeria,9.4,9.3,112,23,0.06,61,21,105,28.4,23.5,0.2
3,American Samoa,..,..,..,..,..,2,..,..,..,..,..
4,Andorra,..,..,..,3,..,8,8,..,..,..,..


In [7]:
#clean header names

wv2.columns=wv2.columns.map("_".join)
wv2.head()

,Unnamed: 0_level_0_Unnamed: 0_level_1,Percentage share of income or consumption_2007-18,Prevalence of child malnutrition_2020,Maternal mortality ratio_2017,Under-five mortality rate_2019,"Incidence of HIV, ages 15-49 (per 1,000 uninfected population ages 15-49)_2020",Incidence of tuberculosis_2019,Mortality caused by road traffic injury_2016,Primary completion rate_2018,Vulnerable employment_2019,Unnamed: 10_level_0_2019,Labor productivity_2015-18
0,Afghanistan,..,35.1,638,60,0.08,189,15,86,76.7,89.5,-2
1,Albania,7.5,9.6,15,10,0.05,16,14,102,51.3,51.1,-0.6
2,Algeria,9.4,9.3,112,23,0.06,61,21,105,28.4,23.5,0.2
3,American Samoa,..,..,..,..,..,2,..,..,..,..,..
4,Andorra,..,..,..,3,..,8,8,..,..,..,..


In [8]:
wv2.columns = [x.lower().replace(" ","_").replace(",","").replace("-","_") for x in wv2.columns]
                    
wv2.head()

,unnamed:_0_level_0_unnamed:_0_level_1,percentage_share_of_income_or_consumption_2007_18,prevalence_of_child_malnutrition_2020,maternal_mortality_ratio_2017,under_five_mortality_rate_2019,incidence_of_hiv_ages_15_49_(per_1000_uninfected_population_ages_15_49)_2020,incidence_of_tuberculosis_2019,mortality_caused_by_road_traffic_injury_2016,primary_completion_rate_2018,vulnerable_employment_2019,unnamed:_10_level_0_2019,labor_productivity_2015_18
0,Afghanistan,..,35.1,638,60,0.08,189,15,86,76.7,89.5,-2
1,Albania,7.5,9.6,15,10,0.05,16,14,102,51.3,51.1,-0.6
2,Algeria,9.4,9.3,112,23,0.06,61,21,105,28.4,23.5,0.2
3,American Samoa,..,..,..,..,..,2,..,..,..,..,..
4,Andorra,..,..,..,3,..,8,8,..,..,..,..


In [9]:
wv2.columns

Index(['unnamed:_0_level_0_unnamed:_0_level_1',
       'percentage_share_of_income_or_consumption_2007_18',
       'prevalence_of_child_malnutrition_2020',
       'maternal_mortality_ratio_2017', 'under_five_mortality_rate_2019',
       'incidence_of_hiv_ages_15_49_(per_1000_uninfected_population_ages_15_49)_2020',
       'incidence_of_tuberculosis_2019',
       'mortality_caused_by_road_traffic_injury_2016',
       'primary_completion_rate_2018', 'vulnerable_employment_2019',
       'unnamed:_10_level_0_2019', 'labor_productivity_2015_18'],
      dtype='object')

In [10]:
wv2_df=wv2.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country","unnamed:_10_level_0_2019":"contributing_family_workers_and_own_account_workers_female",\
                           'incidence_of_hiv_ages_15_49_(per_1000_uninfected_population_ages_15_49)_2020':'incidence_of_hiv_ages_15_49_per1000_2020', 'labor_productivity_2015_18': 'labor_productivity_growth_2015_18'})
wv2_df.head()

,country,percentage_share_of_income_or_consumption_2007_18,prevalence_of_child_malnutrition_2020,maternal_mortality_ratio_2017,under_five_mortality_rate_2019,incidence_of_hiv_ages_15_49_per1000_2020,incidence_of_tuberculosis_2019,mortality_caused_by_road_traffic_injury_2016,primary_completion_rate_2018,vulnerable_employment_2019,contributing_family_workers_and_own_account_workers_female,labor_productivity_growth_2015_18
0,Afghanistan,..,35.1,638,60,0.08,189,15,86,76.7,89.5,-2
1,Albania,7.5,9.6,15,10,0.05,16,14,102,51.3,51.1,-0.6
2,Algeria,9.4,9.3,112,23,0.06,61,21,105,28.4,23.5,0.2
3,American Samoa,..,..,..,..,..,2,..,..,..,..,..
4,Andorra,..,..,..,3,..,8,8,..,..,..,..


In [11]:
#read sustainability data file

wv3 = pd.read_csv("Resources\WV3.csv",header=[0,3])
wv3.head()

,Unnamed: 0_level_0,People using safely managed drinking water services,People using safely managed sanitation services,Access to electricity,Renewable energy consumption,Expenditures for R&D,Urban population living in slums,Ambient PM2.5 air pollution,Adjusted net savings,Carbon dioxide emissions,Nationally protected terrestrial and marine areas,Intentional homicides,Internet use
,Unnamed: 0_level_1,2017,2017,2017,2015,2015,2014,2016,2017,2014,2018,2015,2017
0,Afghanistan,..,..,97.7,18.4,..,62.7,56.3,10.1,0.3,0.1,9.8,13.5
1,Albania,70,39.9,100,38.6,..,..,18.2,-0.7,1.9,13.5,2.2,71.8
2,Algeria,..,17.7,100,0.1,0.5,..,39.9,21.2,3.7,7.1,1.4,47.7
3,American Samoa,12.6,..,..,0.9,..,..,12.4,..,..,8.7,7.2,..
4,Andorra,90.6,100,100,19.7,..,..,10.3,..,5.8,26.7,0,91.6


In [12]:
#clean header names

wv3.columns=wv3.columns.map("_".join)
wv3.head()

,Unnamed: 0_level_0_Unnamed: 0_level_1,People using safely managed drinking water services_2017,People using safely managed sanitation services_2017,Access to electricity_2017,Renewable energy consumption_2015,Expenditures for R&D_2015,Urban population living in slums_2014,Ambient PM2.5 air pollution_2016,Adjusted net savings_2017,Carbon dioxide emissions_2014,Nationally protected terrestrial and marine areas_2018,Intentional homicides_2015,Internet use_2017
0,Afghanistan,..,..,97.7,18.4,..,62.7,56.3,10.1,0.3,0.1,9.8,13.5
1,Albania,70,39.9,100,38.6,..,..,18.2,-0.7,1.9,13.5,2.2,71.8
2,Algeria,..,17.7,100,0.1,0.5,..,39.9,21.2,3.7,7.1,1.4,47.7
3,American Samoa,12.6,..,..,0.9,..,..,12.4,..,..,8.7,7.2,..
4,Andorra,90.6,100,100,19.7,..,..,10.3,..,5.8,26.7,0,91.6


In [13]:
wv3.columns = [x.lower().replace(" ","_").replace("&","") for x in wv3.columns]
                    
wv3.head()

,unnamed:_0_level_0_unnamed:_0_level_1,people_using_safely_managed_drinking_water_services_2017,people_using_safely_managed_sanitation_services_2017,access_to_electricity_2017,renewable_energy_consumption_2015,expenditures_for_rd_2015,urban_population_living_in_slums_2014,ambient_pm2.5_air_pollution_2016,adjusted_net_savings_2017,carbon_dioxide_emissions_2014,nationally_protected_terrestrial_and_marine_areas_2018,intentional_homicides_2015,internet_use_2017
0,Afghanistan,..,..,97.7,18.4,..,62.7,56.3,10.1,0.3,0.1,9.8,13.5
1,Albania,70,39.9,100,38.6,..,..,18.2,-0.7,1.9,13.5,2.2,71.8
2,Algeria,..,17.7,100,0.1,0.5,..,39.9,21.2,3.7,7.1,1.4,47.7
3,American Samoa,12.6,..,..,0.9,..,..,12.4,..,..,8.7,7.2,..
4,Andorra,90.6,100,100,19.7,..,..,10.3,..,5.8,26.7,0,91.6


In [14]:
wv3_df=wv3.rename(columns ={"unnamed:_0_level_0_unnamed:_0_level_1":"country", "ambient_pm2.5_air_pollution_2016":"ambient_pm2_5_air_pollution_2016"})
wv3_df.head()

,country,people_using_safely_managed_drinking_water_services_2017,people_using_safely_managed_sanitation_services_2017,access_to_electricity_2017,renewable_energy_consumption_2015,expenditures_for_rd_2015,urban_population_living_in_slums_2014,ambient_pm2_5_air_pollution_2016,adjusted_net_savings_2017,carbon_dioxide_emissions_2014,nationally_protected_terrestrial_and_marine_areas_2018,intentional_homicides_2015,internet_use_2017
0,Afghanistan,..,..,97.7,18.4,..,62.7,56.3,10.1,0.3,0.1,9.8,13.5
1,Albania,70,39.9,100,38.6,..,..,18.2,-0.7,1.9,13.5,2.2,71.8
2,Algeria,..,17.7,100,0.1,0.5,..,39.9,21.2,3.7,7.1,1.4,47.7
3,American Samoa,12.6,..,..,0.9,..,..,12.4,..,..,8.7,7.2,..
4,Andorra,90.6,100,100,19.7,..,..,10.3,..,5.8,26.7,0,91.6


In [15]:
#drop empty rows

wv1_df.drop(wv1.index[227:400],inplace=True)
wv2_df.drop(wv2.index[226:400],inplace=True)
wv3_df.drop(wv3.index[226:400],inplace=True)

wv1_df

,country,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,ppp_gni_per_capita_2019,gross_domestic_product_growth_2019,per_capita_gdp_growth_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,..,..,..,..,2.2,2.5
4,Andorra,0.1,0.5,164,..,..,..,..,1.8,1.7
...,...,...,...,...,...,...,...,...,...,...
222,Sub-Saharan Africa,"1,107.00","21,754.50",45,"1,720.80","1,555","4,207.00","3,801",2.3,-0.4
223,Low income,668.5,"12,870.10",43,547.9,820,"1,637.70","2,450",4,1.3
224,Lower middle income,"2,913.40","22,578.80",131,"6,340.10","2,176","19,697.90","6,761",4,2.5
225,Upper middle income,"2,855.90","59,108.00",49,"25,884.40","9,064","49,338.80","17,276",3.6,2.9


In [16]:
#drop invalid cell data

wv1_df1=wv1_df.replace(to_replace="..", value="NaN")
wv2_df1=wv2_df.replace(to_replace="..", value="NaN")
wv3_df1=wv3_df.replace(to_replace="..", value="NaN")

wv1_df1

,country,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,ppp_gni_per_capita_2019,gross_domestic_product_growth_2019,per_capita_gdp_growth_2019
0,Afghanistan,38,652.9,57,20,530,83.3,"2,190",3.9,1.5
1,Albania,2.9,28.8,105,14.9,"5,220",41.3,"14,450",2.2,2.7
2,Algeria,43.1,"2,381.70",18,172.5,"4,010",504.8,"11,720",0.8,-1.1
3,American Samoa,0.1,0.2,277,NaN,NaN,NaN,NaN,2.2,2.5
4,Andorra,0.1,0.5,164,NaN,NaN,NaN,NaN,1.8,1.7
...,...,...,...,...,...,...,...,...,...,...
222,Sub-Saharan Africa,"1,107.00","21,754.50",45,"1,720.80","1,555","4,207.00","3,801",2.3,-0.4
223,Low income,668.5,"12,870.10",43,547.9,820,"1,637.70","2,450",4,1.3
224,Lower middle income,"2,913.40","22,578.80",131,"6,340.10","2,176","19,697.90","6,761",4,2.5
225,Upper middle income,"2,855.90","59,108.00",49,"25,884.40","9,064","49,338.80","17,276",3.6,2.9


In [17]:
wv1_df2=wv1_df1.replace(",", "", regex=True)
wv2_df2=wv2_df1.replace(",", "", regex=True)
wv3_df2=wv3_df1.replace(",", "", regex=True)

wv3_df2

,country,people_using_safely_managed_drinking_water_services_2017,people_using_safely_managed_sanitation_services_2017,access_to_electricity_2017,renewable_energy_consumption_2015,expenditures_for_rd_2015,urban_population_living_in_slums_2014,ambient_pm2_5_air_pollution_2016,adjusted_net_savings_2017,carbon_dioxide_emissions_2014,nationally_protected_terrestrial_and_marine_areas_2018,intentional_homicides_2015,internet_use_2017
0,Afghanistan,NaN,NaN,97.7,18.4,NaN,62.7,56.3,10.1,0.3,0.1,9.8,13.5
1,Albania,70,39.9,100,38.6,NaN,NaN,18.2,-0.7,1.9,13.5,2.2,71.8
2,Algeria,NaN,17.7,100,0.1,0.5,NaN,39.9,21.2,3.7,7.1,1.4,47.7
3,American Samoa,12.6,NaN,NaN,0.9,NaN,NaN,12.4,NaN,NaN,8.7,7.2,NaN
4,Andorra,90.6,100,100,19.7,NaN,NaN,10.3,NaN,5.8,26.7,0,91.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Sub-Saharan Africa,27.3,18.7,44.7,70.1,NaN,55.3,44.4,-1.2,0.9,15.7,9.4,25.4
222,Low income,26.7,NaN,40.7,76.2,NaN,68.4,42.5,2,0.3,12.9,7.2,15.1
223,Lower middle income,53.7,NaN,84.6,40.6,0.6,33.5,67.3,14.2,1.5,11.9,4.9,34.9
224,Upper middle income,NaN,59.6,99.2,14.4,1.5,23.3,36.9,13.7,6.2,12.9,6.7,56.7


In [18]:
wv1_df2

,country,population_2019,surface_area_2019,population_density_2019,gross_national_income_atlas_method_2019,gross_national_income_per_capita_atlas_method_2019,purchasing_power_parity_gross_national_income_2019,ppp_gni_per_capita_2019,gross_domestic_product_growth_2019,per_capita_gdp_growth_2019
0,Afghanistan,38,652.9,57,20,530,83.3,2190,3.9,1.5
1,Albania,2.9,28.8,105,14.9,5220,41.3,14450,2.2,2.7
2,Algeria,43.1,2381.70,18,172.5,4010,504.8,11720,0.8,-1.1
3,American Samoa,0.1,0.2,277,NaN,NaN,NaN,NaN,2.2,2.5
4,Andorra,0.1,0.5,164,NaN,NaN,NaN,NaN,1.8,1.7
...,...,...,...,...,...,...,...,...,...,...
222,Sub-Saharan Africa,1107.00,21754.50,45,1720.80,1555,4207.00,3801,2.3,-0.4
223,Low income,668.5,12870.10,43,547.9,820,1637.70,2450,4,1.3
224,Lower middle income,2913.40,22578.80,131,6340.10,2176,19697.90,6761,4,2.5
225,Upper middle income,2855.90,59108.00,49,25884.40,9064,49338.80,17276,3.6,2.9


In [19]:
wv1_df2['population_2019'] = round(1000000 * wv1_df2['population_2019'].astype(float), 3)
wv1_df2['surface_area_2019'] = round(1000 * wv1_df2['surface_area_2019'].astype(float), 3)
wv1_df2['population_density_2019'] = round(wv1_df2['population_density_2019'].astype(float), 3)
wv1_df2['gross_national_income_atlas_method_2019'] = round(1000000000 * wv1_df2['gross_national_income_atlas_method_2019'].astype(float), 3)
wv1_df2['gross_national_income_per_capita_atlas_method_2019'] = round(wv1_df2['gross_national_income_per_capita_atlas_method_2019'].astype(float), 3)
wv1_df2['purchasing_power_parity_gross_national_income_2019'] = round(1000000000 * wv1_df2['purchasing_power_parity_gross_national_income_2019'].astype(float), 3)
wv1_df2['ppp_gni_per_capita_2019'] = round(wv1_df2['ppp_gni_per_capita_2019'].astype(float), 3)
wv1_df2['gross_domestic_product_growth_2019'] = round(0.01 * wv1_df2['gross_domestic_product_growth_2019'].astype(float), 3)
wv1_df2['per_capita_gdp_growth_2019'] = round(0.01 * wv1_df2['per_capita_gdp_growth_2019'].astype(float), 3)

In [20]:
wv2_df2['percentage_share_of_income_or_consumption_2007_18'] = round(0.01 * wv2_df2['percentage_share_of_income_or_consumption_2007_18'].astype(float), 3)
wv2_df2['prevalence_of_child_malnutrition_2020'] = round(0.01 * wv2_df2['prevalence_of_child_malnutrition_2020'].astype(float), 3)
wv2_df2['maternal_mortality_ratio_2017'] = round(0.00001 * wv2_df2['maternal_mortality_ratio_2017'].astype(float), 3)
wv2_df2['under_five_mortality_rate_2019'] = round(0.001 * wv2_df2['under_five_mortality_rate_2019'].astype(float), 3)
wv2_df2['incidence_of_hiv_ages_15_49_per1000_2020'] = round(0.001 * wv2_df2['incidence_of_hiv_ages_15_49_per1000_2020'].astype(float), 3)
wv2_df2['incidence_of_tuberculosis_2019'] = round(0.00001 * wv2_df2['incidence_of_tuberculosis_2019'].astype(float), 3)
wv2_df2['mortality_caused_by_road_traffic_injury_2016'] = round(0.00001 * wv2_df2['mortality_caused_by_road_traffic_injury_2016'].astype(float), 3)
wv2_df2['primary_completion_rate_2018'] = round(0.01 * wv2_df2['primary_completion_rate_2018'].astype(float), 3)
wv2_df2['vulnerable_employment_2019'] = round(0.01 * wv2_df2['vulnerable_employment_2019'].astype(float), 3)
wv2_df2['contributing_family_workers_and_own_account_workers_female'] = round(0.01 * wv2_df2['contributing_family_workers_and_own_account_workers_female'].astype(float), 3)
wv2_df2['labor_productivity_growth_2015_18'] = round(0.01 * wv2_df2['labor_productivity_growth_2015_18'].astype(float), 3)

In [21]:
wv3_df2['people_using_safely_managed_drinking_water_services_2017'] = round(0.01 * wv3_df2['people_using_safely_managed_drinking_water_services_2017'].astype(float), 3)
wv3_df2['people_using_safely_managed_sanitation_services_2017'] = round(0.01 * wv3_df2['people_using_safely_managed_sanitation_services_2017'].astype(float), 3)
wv3_df2['access_to_electricity_2017'] = round(0.01 * wv3_df2['access_to_electricity_2017'].astype(float), 3)
wv3_df2['renewable_energy_consumption_2015'] = round(0.01 * wv3_df2['renewable_energy_consumption_2015'].astype(float), 3)
wv3_df2['expenditures_for_rd_2015'] = round(0.01 * wv3_df2['expenditures_for_rd_2015'].astype(float), 3)
wv3_df2['urban_population_living_in_slums_2014'] = round(0.01 * wv3_df2['urban_population_living_in_slums_2014'].astype(float), 3)
wv3_df2['ambient_pm2_5_air_pollution_2016'] = round(wv3_df2['ambient_pm2_5_air_pollution_2016'].astype(float), 3)
wv3_df2['adjusted_net_savings_2017'] = round(0.01 * wv3_df2['adjusted_net_savings_2017'].astype(float), 3)
wv3_df2['carbon_dioxide_emissions_2014'] = round(wv3_df2['carbon_dioxide_emissions_2014'].astype(float), 3)
wv3_df2['nationally_protected_terrestrial_and_marine_areas_2018'] = round(0.01 * wv3_df2['nationally_protected_terrestrial_and_marine_areas_2018'].astype(float), 3)
wv3_df2['intentional_homicides_2015'] = round(0.00001 * wv3_df2['intentional_homicides_2015'].astype(float), 3)
wv3_df2['internet_use_2017'] = round(0.01 * wv3_df2['internet_use_2017'].astype(float), 3)

In [22]:
#create postgres connection

rds_connection_string = username + ":" + password + "@localhost:5432/world_development_indicators"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [23]:
engine.table_names()

<ipython-input-23-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['economy_size', 'ending_poverty', 'promoting_sustainability']

In [24]:
#write data to economy_size table

wv1_df2.to_sql(name='economy_size', con=engine, if_exists='append', index=False)

In [25]:
#write data to ending_poverty table

wv2_df2.to_sql(name='ending_poverty', con=engine, if_exists='append', index=False)

In [26]:
#write data to promoting_sustainability table

wv3_df2.to_sql(name='promoting_sustainability', con=engine, if_exists='append', index=False)